In [34]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [35]:
test = np.loadtxt(r'C:\Users\pc\PycharmProjects\Netology\Classwork\CV\002\data\digit\test.csv', delimiter=',', skiprows=1)
train = np.loadtxt(r'C:\Users\pc\PycharmProjects\Netology\Classwork\CV\002\data\digit\train.csv', delimiter=',', skiprows=1)

y = train[:, 0]
Y_val = y[0:28000:]

train_img = np.reshape(train[:, 1:], (len(train[:, 1:]), 28, 28))
test_img = np.reshape(test[:,:], (len(test[:,:]), 28, 28))

X = train_img[::].reshape(-1, 28 * 28).astype(np.float32)
X_test = test[::].reshape(-1, 28 * 28).astype(np.float32)

#этот код для выборки как в примере у вас random 10000 samples

# choices = np.random.choice(train_img.shape[0], 10000)

# y3 = train_label[choices]
# X3 = train_img[choices].reshape(-1, 28 * 28).astype(np.float32)


In [36]:
# центрируем данные
X_mean = X.mean(axis=0)
X -= X_mean

# X_mean = X3.mean(axis=0)
# X3 -= X_mean

X_test = X_test.astype('float32')
X_test_mean = X_test.mean(axis=0)
X_test -= X_test_mean

In [37]:
cov = np.dot(X.T, X) / X.shape[0]
cov2 = np.dot(X_test.T, X_test) / X_test.shape[0]
# cov3 = np.dot(X3.T, X3) / X3.shape[0]


# print(x)

In [38]:
U, S, _ = np.linalg.svd(cov)
U2, S2, _ = np.linalg.svd(cov2)
# U3, S3, _ = np.linalg.svd(cov3)

In [39]:
S_cumsum = np.cumsum(S) / np.sum(S)
S_cumsum2 = np.cumsum(S2) / np.sum(S2)
# S_cumsum3 = np.cumsum(S3) / np.sum(S3)


In [40]:
for i in range(5):
    print('[%03d] %.3f' % (i, S_cumsum[i]))

[000] 0.097
[001] 0.169
[002] 0.231
[003] 0.284
[004] 0.333


In [41]:
S_thr = 0.60  # задаем порог для накопленной суммы собственных значений

# определяем необходимое число компонент для заданного порога
n_comp = np.argmax(np.where(S_cumsum > S_thr, 1, 0))
n_comp2 = np.argmax(np.where(S_cumsum2 > S_thr, 1, 0))
n_comp3 = np.argmax(np.where(S_cumsum2 > S_thr, 1, 0))


In [42]:
X_test_reduced = np.dot(X_test, U[:, :n_comp])
Xrot_reduced = np.dot(X, U2[:, :n_comp2])
# X3_reduced = np.dot(X3, U3[:, :n_comp3])

In [43]:
# all samples
x_train,x_val, y_train,y_val = train_test_split(Xrot_reduced, y, test_size = 0.1, random_state=1)

# random 10000 samples
# x_train, x_val, y_train, y_val = train_test_split(X3_reduced, y3, test_size =0.1 , random_state=1)

In [44]:
lr = LogisticRegression(random_state=1,n_jobs=-1,penalty='l2')
lr.fit(x_train,y_train)
y_pred = lr.predict(X_test_reduced)

print(y_pred.shape)

print('accuracy',accuracy_score(Y_val,y_pred))
y_pred = y_pred.astype(int)

(28000,)
accuracy 0.10160714285714285


In [45]:
with open('submit.txt', 'w') as dst:
    dst.write('ImageId,Label\n')
    for i, p in enumerate(y_pred, 1):
        dst.write('%s,%s\n' % (i, p))

